# Crunchbase API v4 set up - Draft Code

**From CrunchBase**:
- How to Use the API - https://data.crunchbase.com/docs/using-the-api
- How to find which fields and data are available via our API - https://data.crunchbase.com/docs/available-data
- Swaggerhub (contains API calls and responses) - https://app.swaggerhub.com/apis-docs/Crunchbase/crunchbase-enterprise_api/#/Entity/get_entities_organizations__entity_id_
- API Reference - https://data.crunchbase.com/reference


### Next Steps (8/10):
- Query 1: Organization --> Financial Summary, Funding Rounds, Board Members
- Query 2: Investor --> Board Affiliations, Partner Funding Rounds
- Query 3: Match-making (Corporate BoD <> Partner Investors)
- Push to Google Sheets: https://towardsdatascience.com/how-to-integrate-google-sheets-and-jupyter-notebooks-c469309aacea
- Create crunchbase-p1 module with generalized queries and interactive input

In [4]:
#userkey

### Import all necessary packages

In [5]:
# Requests allows you to send HTTP/1.1 requests extremely easily.
# There’s no need to manually add query strings to your URLs, or to form-encode your POST data. 
# Keep-alive and HTTP connection pooling are 100% automatic, thanks to urllib3.
import requests
import json
import pandas as pd
import csv
from pandas import json_normalize 
from operator import itemgetter
from user_key import userkey # Pulls in P1s Crunchbase API user key

### helper functions

In [4]:
def url_count(query, query_type): 
    # POST method with API URL, query_type as a parameter, and passing query as json.
    # https://www.w3schools.com/python/ref_requests_post.asp
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params = userkey , json = query)
    #print(r.text)
    #print(r.url)
    result = json.loads(r.text)
    count = result["count"]
    return count

def url_extraction(query, query_type):    
    # Create global raw variable
    global raw   
    # POST method with API URL, query_type as a parameter, and passing query as json.
    # https://www.w3schools.com/python/ref_requests_post.asp
    r = requests.post("https://api.crunchbase.com/api/v4/searches/" + query_type, params = userkey , json = query)
    result = json.loads(r.text)
    normalized_raw = json_normalize(result['entities'])
    # Append normalized entity results to global raw variable
    raw = raw.append(normalized_raw, ignore_index=True)

def autocompletes(query, collection_ids_list=None, limit=None):
    '''
    Suggests matching Identifier entities based on the query and entity_def_ids provided.
    
    QUERY
    Value to perform the autocomplete search with.
    
    COLLECTION_IDS_LIST
    A comma separated list of collection ids to search against. 
    Leaving this blank means it will search across all identifiers. 
    Entity defs can be constrained to specific facets by providing them as facet collections. 
    Relationship collections will resolve to their underlying entity def.
    Collection ids are: organizations, people, funding_rounds, acquisitions, investments,
    events, press_references, funds, event_appearances, ipos, ownerships, categories, 
    category_groups, locations, jobs
    
    LIMIT
    Number of results to retrieve; default = 10, max = 25
    '''
    params = {**userkey, "query": query}
    if collection_ids_list and type(collection_ids_list) == list:
        params.update({"collection_ids": collection_ids_list})
    if limit and type(limit) == int:
        params.update({"limit": limit})
    r = requests.get("https://api.crunchbase.com/api/v4/autocompletes", params = params)
    result = json.loads(r.text)
    normalized_result = json_normalize(result["entities"])
    dataframe = pd.DataFrame.from_dict(normalized_result)
    return dataframe

def makequery_jobs(uuid_list, limit=1000):
    '''
    Job Search: Current Board/Executive Affiliations
    - Person includes list of `uuid` values
    - The job title is current (`is_current == True`)
    - Excludes `employee` level jobs
    '''
    query = {
        "field_ids": [
            "entity_def_id",
            "identifier",
            "job_type",
            "name",
            "organization_identifier",
            "person_identifier",
            "short_description",
            "is_current",
            "started_on",
            "ended_on",
            "title",
            "updated_at",
            "uuid"],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "person_identifier",
                "operator_id": "includes",
                "values": uuid_list
            },
            {
                "type": "predicate",
                "field_id": "is_current",
                "operator_id": "eq",
                "values": ["true"]
            },
            {
                "type": "predicate",
                "field_id": "job_type",
                "operator_id": "not_includes",
                "values": ["employee"]
            }]
    }
    return query

def whoKnows(name, df, person_first=True):
    '''
    This function takes in a name string and a dataframe generated by the url_extraction(query, "jobs") function.
    
    Returns a string that is a concatenated list of unique names mapped to their companies, excluding the rows of the named person.
    
    This is a Pledge 1% action that helps us see who knows a specific Boardroom Ally and how, pulled from the Boardroom Allies affiliations dataframe.
    
    If person_first is True (default), the output will aggregate by person. 
    matches_str = 'Name1 (Company1, Company), Name2 (Company2), Name3 (Company1), ...'
    
    If person_first is False, the output will aggregate by company
    matches_str = 'Company1 (Name1, Name2, Name3), Company2 (Name1), Company3 (Name2, Name3), ...'
    '''
    # Don't start if the input name search comes up with nothing.
    if df["properties.organization_identifier.value"][df["properties.person_identifier.value"] == name] is None:
        return "There's no one in this list by that name. Check for typos!"
    
    # Make a list of all unique companies affiliated with input name
    company_matches = set(df["properties.organization_identifier.value"][df["properties.person_identifier.value"] == name].to_list())
    
    # Create matches dataframe, filtering by those that match the unique company list. 
    matches_df = df[df["properties.organization_identifier.value"].isin(company_matches)]
    
    # Remove the person from the results, to remove situations like: "Byron Deeter" knowing "Byron Deeter"
    matches_df = matches_df[matches_df["properties.person_identifier.value"] != name].sort_values(["properties.organization_identifier.value"])
    
    # Create intermediate dictionary which will de-dupe based on `person_first` value
    person_matches = matches_df["properties.person_identifier.value"].to_list()
    company_matches = matches_df["properties.organization_identifier.value"].to_list()
    matches_dict = {}
    
    if person_first:
        # Aggregate by name.
        for i in range(len(person_matches)):
            # If person already in dictionary
            if person_matches[i] in matches_dict.keys():
                # Append company to list
                matches_dict[person_matches[i]].append(company_matches[i])
            else:
                # Add person to dictionary
                matches_dict[person_matches[i]] = [company_matches[i]]
    
    if not person_first:
        # Aggregate by company.
        for i in range(len(person_matches)):
            # If company already in dictionary
            if company_matches[i] in matches_dict.keys():
                # Append person to list
                matches_dict[company_matches[i]].append(person_matches[i])
            else:
                # Add company to dictionary
                matches_dict[company_matches[i]] = [person_matches[i]]    
    
    # Create output string
    matches_str = ""
    for key, value in matches_dict.items():
        # Start returned string
        matches_str += key + " ("
        # Start empty string for loop
        forloop_str = ""
        for i, item in enumerate(matches_dict[key]):
            if i == len(matches_dict[key]) - 1:
                forloop_str += item + ")"
                continue
            forloop_str += item + ", "
        matches_str += forloop_str + ", "
    # Remove extra space and comma
    matches_str = matches_str[:-2]
    
    return matches_str

def makequery_jobs_board_of_org(uuid_list, limit=1000):
    '''
    Job Search: Current Board Affiliations
    - Organization includes list of `uuid` values
    - Excludes `employee` and `executive` level jobs
    '''
    query = {
        "field_ids": [
            "entity_def_id",
            "identifier",
            "job_type",
            "name",
            "organization_identifier",
            "person_identifier",
            "short_description",
            "is_current",
            "started_on",
            "ended_on",
            "title",
            "updated_at",
            "uuid"],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "organization_identifier",
                "operator_id": "includes",
                "values": uuid_list
            },
            {
                "type": "predicate",
                "field_id": "job_type",
                "operator_id": "not_includes",
                "values": ["employee", "executive"]
            }]
    }
    return query

def makequery_jobs_exec(uuid_list, limit=1000):
    '''
    Job Search: Current Executive Roles
    - Person includes list of `uuid` values
    - The job title is current (`is_current == True`)
    - Excludes `employee`, `board_member`, `advisor`, and `board_observer` job types
    '''
    query = {
        "field_ids": [
            "entity_def_id",
            "employee_featured_order",
            "identifier",
            "job_type",
            "name",
            "organization_identifier",
            "person_identifier",
            "short_description",
            "is_current",
            "started_on",
            "ended_on",
            "title",
            "updated_at",
            "uuid"],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "person_identifier",
                "operator_id": "includes",
                "values": uuid_list
            },
            {
                "type": "predicate",
                "field_id": "is_current",
                "operator_id": "eq",
                "values": ["true"]
            },
            {
                "type": "predicate",
                "field_id": "job_type",
                "operator_id": "not_includes",
                "values": ["employee", "board_member", "advisor", "board_observer"]
            }]
    }
    return query

# UUIDs

### Boardroom Allies - Get UUIDs

In [12]:
boardroom_ally_targets = pd.read_csv("input/Boardroom Allies Tracker - job_search_terms.csv").CONCAT.to_list()
boardroom_ally_targets.sort()
uuid_boardroom_ally_targets = []
counter = 0
for person in boardroom_ally_targets:
    counter += 1
    print("****************************Searching............{}\n{}".format(counter, person.upper()))
    found = autocompletes(person, ["people"], limit=1)
    uuid_boardroom_ally_targets.append(found["identifier.uuid"][0])
    print("Found............{}\nDESCRIPTION: {}\n".format(found["identifier.value"][0].upper(),found["short_description"][0]))

****************************Searching............1
AJAY AGARWAL BAIN
Found............AJAY AGARWAL
DESCRIPTION: Ajay Aqarwal leads the Bay Area offices for Bain Capital Ventures, where he focuses on early-stage application software and SaaS investing. ...

****************************Searching............2
ALEX BARD REDPOINT
Found............ALEX BARD
DESCRIPTION: Alex Bar is the Managing Director at Redpoint Ventures and a Board Member at Coder.

****************************Searching............3
ANDREW CHEN ANDREESSEN
Found............ANDREW CHEN
DESCRIPTION: Andrew Chen is a General Partner at Andreessen Horowitz, focusing on consumer technology. Previously, he led the the Rider Growth teams at U...

****************************Searching............4
ANDY WEISSMAN USVP
Found............ANDY WEISSMAN
DESCRIPTION: Andy Weissman is a partner at Union Square Ventures. Andy began his career in the Internet in the mid-90s. Prior to joining USV, in 2007 he ...

****************************

Found............JASON MENDELSON
DESCRIPTION: Jason Mendelson is a co-founder of Foundry Group and has over a decade of experience in the venture capital and technology industries in a m...

****************************Searching............34
JEFF EPSTEIN BESSEMER
Found............JEFF EPSTEIN
DESCRIPTION: Jeff Epstein is an Operating Partner at Bessemer Venture Partners, where he specializes in Marketplaces and Cloud Computing.

Jeff is the ...

****************************Searching............35
JEFF JORDAN ANDREESSEN
Found............JEFF JORDAN
DESCRIPTION: Jeff Jordan is a Managing Partner and General Partner at the venture capital firm Andreessen Horowitz where he serves on the board of 500px,...

****************************Searching............36
JEREMY LEVINE BESSEMER
Found............JEREMY LEVINE
DESCRIPTION: By leading early stage investments in Yelp, Pinterest and LinkedIn, Jeremy Levine has earned a reputation for spotting fledgling stars while...

***********************

Found............PUNEET AGARWAL
DESCRIPTION: Puneet brings a strong mix of operational and investment experience to his partner role at True. He began his career as a product manager at...

****************************Searching............66
RAJEEV BATRA MAYFIELD
Found............RAJEEV BATRA
DESCRIPTION: Rajeev Batra is a Partner at Mayfield helps lead the firm's Enterprise practice and invests in Cloud and SaaS. He is a former enterprise sof...

****************************Searching............67
RAYMOND TONSING CAFFEINATED
Found............RAYMOND TONSING
DESCRIPTION: Raymond Tonsing is a San Francisco based venture capitalist and entrepreneur. He founded Caffeinated Capital, an early stage venture capital...

****************************Searching............68
RICHARD WONG ACCEL
Found............RICHARD WONG
DESCRIPTION: Rich Wong joined Accel as a partner in 2006. Rich led Accel's investments and currently serves on the Boards of Atlassian (TEAM), Checkr, In...

******************

### P1 Equity Companies - Get UUIDs

In [16]:
p1_equity_companies = pd.read_csv("input/Boardroom Allies Tracker - company_search_terms.csv").Company.to_list()
p1_equity_companies.sort()
uuid_p1_equity_companies = []
p1_equity_companies_val = []
counter = 0
for company in p1_equity_companies:
    counter += 1
    print("****************************Searching............{}\n{}".format(counter, company.upper()))
    found = autocompletes(company, ["organizations"], limit=1)
    p1_equity_companies_val.append(found["identifier.value"][0])
    uuid_p1_equity_companies.append(found["identifier.uuid"][0])
    print("Found............{}\nDESCRIPTION: {}\n".format(found["identifier.value"][0].upper(),found["short_description"][0]))

****************************Searching............1
APPFIRE
Found............APPFIRE
DESCRIPTION: Appfire is a provider of Atlassian Enterprise products and services.

****************************Searching............2
ATLASSIAN
Found............ATLASSIAN
DESCRIPTION: Atlassian provides collaboration, development, and issue tracking software for teams.

****************************Searching............3
AUTH0
Found............AUTH0
DESCRIPTION: Auth0 operates a cloud-based identity platform for developers.

****************************Searching............4
CANVA
Found............CANVA
DESCRIPTION: Canva is an online design and publishing platform that provides user friendly design tools for non-designers.

****************************Searching............5
CODE42
Found............CODE42
DESCRIPTION: Code42 provides data loss protection, visibility, and recovery solutions.

****************************Searching............6
COINBASE
Found............COINBASE
DESCRIPTION: Coinbase is a d

# Boardroom Allies

### Boardroom Allies - Current Board Affiliations

In [17]:
query = makequery_jobs(uuid_boardroom_ally_targets)
raw = pd.DataFrame() # Global raw variable
comp_count = url_count(query, "jobs") 
url_extraction(query, "jobs")
boardroom_allies_board_affiliations = raw[["properties.organization_identifier.value",
                    "properties.person_identifier.value", 
                    "properties.title", 
                    "properties.job_type", 
                    "properties.started_on.value",
                    "properties.updated_at"]].sort_values(["properties.organization_identifier.value"])
boardroom_allies_board_affiliations.to_csv("output/BA_board_affiliations.csv")
boardroom_allies_board_affiliations.head()

,properties.organization_identifier.value,properties.person_identifier.value,properties.title,properties.job_type,properties.started_on.value,properties.updated_at
334,140 Proof,Ronald Conway,Member of the Board of Advisory,advisor,NaN,2018-02-13T01:25:41Z
676,23andMe,Roelof Botha,Board Member,board_member,2017-09-01,2019-03-30T11:13:50Z
754,3Com,Sanford Robertson,Member of the Board of Directors,board_member,NaN,2018-02-13T01:34:39Z
352,3D Robotics,Jason Mendelson,Board Member,board_member,NaN,2018-02-13T11:48:53Z
89,42Floors,Jeremy Levine,Member of the Board of Directors,board_member,2012-05-01,2018-02-13T01:20:13Z


### Boardroom Allies - Matchmaking

In [18]:
# Read in results from previous board affiliations search
bra_affiliations = pd.read_csv("output/BA_board_affiliations.csv")

# Create sub-dataframe that just has affiliations associated with P1 Equity Company
p1_affiliations = bra_affiliations[bra_affiliations["properties.organization_identifier.value"].isin(p1_equity_companies_val)]

# Create list of unique names for input into whoKnows function
names = list(set(bra_affiliations["properties.person_identifier.value"].to_list()))
names.sort()


knows = []
knows_by_company = []
knows_by_p1 = []

for person in names:
    #Call to helper function whoKnows
    knows.append(whoKnows(person, bra_affiliations))
    knows_by_company.append(whoKnows(person, bra_affiliations, False))
    
    # Also source list of p1 companies they're affiliated with
    p1_string = ""
    knows_by_p1_list = list(set(p1_affiliations["properties.organization_identifier.value"][p1_affiliations["properties.person_identifier.value"] == person].to_list()))
    if knows_by_p1_list != []:
        knows_by_p1_list.sort()
        # Add them to string
        for company in knows_by_p1_list:
            p1_string += company + ", "
        # Remove trailing comma
        p1_string = p1_string[:-2]
    knows_by_p1.append(p1_string)

d = {'Name':names, 'Knows':knows, 'Knows_ByCompany':knows_by_company, "KnowsWhich_P1Company":knows_by_p1}
bra_affiliations_matches = pd.DataFrame(d, columns = ["Name","Knows","Knows_ByCompany","KnowsWhich_P1Company"])
bra_affiliations_matches.to_csv("output/BA_matchmaking.csv")
bra_affiliations_matches.head()

,Name,Knows,Knows_ByCompany,KnowsWhich_P1Company
0,Ajay Agarwal,"Enrique Salem (Bain Capital Ventures, Clari), ...","Bain Capital Ventures (Enrique Salem), Clari (...",Optimizely
1,Alex Bard,,,
2,Andrew Chen,"Susan Su (500 Startups), Ben Horowitz (Andrees...","500 Startups (Susan Su), Andreessen Horowitz (...",Dropbox
3,Andy Weissman,David Pakman (YouNow),YouNow (David Pakman),
4,Arne Duncan,"Ryan Hinkle (Pluralsight, Turnitin), Tim Maudl...","Pluralsight (Ryan Hinkle, Tim Maudlin), Turnit...",Pluralsight


# P1 Equity Companies

### P1 Equity Companies - Current/Past Board Affiliations

In [21]:
query = makequery_jobs_board_of_org(uuid_p1_equity_companies)
raw = pd.DataFrame() # Global raw variable
comp_count = url_count(query, "jobs") 
url_extraction(query, "jobs")
p1_companies_board_affiliations = raw[["properties.organization_identifier.value",
                    "properties.person_identifier.value", 
                    "properties.title", 
                    "properties.job_type", 
                    "properties.is_current",
                    "properties.started_on.value",
                    "properties.ended_on.value",
                    "properties.updated_at"]].sort_values(["properties.organization_identifier.value"])
p1_companies_board_affiliations.to_csv("output/p1_companies_board_affiliations.csv")
p1_companies_board_affiliations.head()

,properties.organization_identifier.value,properties.person_identifier.value,properties.title,properties.job_type,properties.is_current,properties.started_on.value,properties.ended_on.value,properties.updated_at
330,Atlassian,Ryan Sweeney,Director,board_member,False,2015-01-01,2017-01-01,2019-09-06T11:03:02Z
46,Atlassian,Steve Sordello,Director,board_member,True,2015-11-01,NaN,2019-09-06T13:26:35Z
307,Atlassian,Murray Demo,Director,board_member,False,2011-12-01,2015-10-01,2019-09-06T13:05:36Z
206,Atlassian,Kirk Bowman,Director,board_member,False,2012-01-01,2016-01-01,2019-09-06T12:19:37Z
112,Atlassian,Richard Wong,Director,board_member,True,2010-07-01,NaN,2019-09-06T13:05:36Z
...,...,...,...,...,...,...,...,...
400,Zuora,Tien Tzuo,Director,board_member,True,2008-01-01,NaN,2019-09-27T15:27:02Z
210,Zuora,Abhishek Agrawal,Director,board_member,False,2015-01-01,2018-01-01,2019-09-27T16:00:55Z
391,Zuora,Shawn Kung,Investor,advisor,True,2016-09-01,NaN,2020-05-23T23:48:00Z
285,appfire,Lyndsay Kerwin,Board Member,board_member,True,2020-06-09,NaN,2020-06-10T13:10:16Z


### P1 Equity Companies - Current/Past Board Affiliations - Get Executive Roles

In [22]:
# Create list of unique UUIDs of board members from previous section
uuid_p1_equity_companies_board = list(set(raw['properties.person_identifier.uuid'].to_list()))

# Split into sublists no larger than 200
chunks = [uuid_p1_equity_companies_board[x:x+200] for x in range(0, len(uuid_p1_equity_companies_board), 200)] 

# For concatenating the results below
frames = []

for i,chunk in enumerate(chunks):
    raw = pd.DataFrame() # Global raw variable
    query = makequery_jobs_exec(chunk)
    url_extraction(query, "jobs")
    frames += [raw[["properties.organization_identifier.value",
                    "properties.person_identifier.value", 
                    "properties.title", 
                    "properties.job_type", 
                    "properties.started_on.value",
                    "properties.updated_at"]].sort_values(["properties.person_identifier.value"])]

# Concatenate the results
result = pd.concat(frames).sort_values(["properties.person_identifier.value"])
result.to_csv("output/p1_companies_board_members_jobs.csv")
result.head()

,properties.organization_identifier.value,properties.person_identifier.value,properties.title,properties.job_type,properties.started_on.value,properties.updated_at
33,Pluralsight,Aaron Skonnard,Co-Founder and CEO,executive,2009-10-01,2020-04-20T04:03:35Z
20,Vulcan Capital,Abhishek Agrawal,Managing Director & Head of Growth Equity,executive,2013-04-01,2018-03-01T22:50:45Z
39,Natural Resources Defense Council,Adrianna Quintero,Senior Attorney & Director of Partner Engagement,executive,NaN,2018-02-13T01:21:30Z
113,Memento Mori,Adriel Lares,Partner,executive,2010-06-01,2018-02-13T01:27:38Z
139,Fastly,Adriel Lares,CFO,executive,2016-05-01,2020-04-15T12:25:14Z


# Draft Functions

### `makequery_org` & `makequery_investors` helper functions

In [ ]:
def makequery_org(uuid, limit=1000):
    '''
    
    '''
    query = {
        "field_ids": [
            "name",
            "website_url",
            "founder_identifiers",
            "categories",
            "category_groups",
            "diversity_spotlights",
            "funding_stage",
            "funding_total",
            "funds_total",
            "last_funding_type",
            "last_funding_at",
            "hub_tags",
            "revenue_range",
            "uuid"],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "uuid",
                "operator_id": "includes",
                "values": uuid
            }]
    }
    return query

def makequery_investors(uuid, limit=1000):
    '''
    
    '''
    query = {
        "field_ids": [
            "name",
            "investor_identifier",
            "organization_identifier"
        ],
        "limit": limit,
        "query": [
            {
                "type": "predicate",
                "field_id": "organization_identifier",
                "operator_id": "includes",
                "values": uuid
            }]
    }
    return query

### Cloud 100 - Organization

In [ ]:
query = makequery_org(uuid_c100_companies)
raw = pd.DataFrame() # Global raw variable
comp_count = url_count(query, "organizations") 
url_extraction(query, "organizations")

In [ ]:
raw[["properties.name","properties.website_url"]].to_csv("cloud100_websites.csv")

In [ ]:
revenue_range = {
    "r_00000000": "Less than $1M",
    "r_00001000": "$1M to $10M",
    "r_00010000": "$10M to $50M",
    "r_00050000": "$50M to $100M",
    "r_00100000": "$100M to $500M",
    "r_00500000": "$500M to $1B",
    "r_01000000": "$1B to $10B",
    "r_10000000": "$10B+"}
master = pd.DataFrame()
master["company"] = raw["properties.identifier.value"]
master["revenue"] = raw["properties.revenue_range"].map(revenue_range)
master["categories"] = raw["properties.categories"].apply(lambda x: 
                                                          list(map(itemgetter('value'), x)
                                                               if isinstance(x, list) 
                                                               else ["Not found"])).apply(lambda x : 
                                                                                          ",".join(map(str, x)))
master.head()

### Cloud 100 - Investors

In [ ]:
query = makequery_investors(uuid_c100_companies)
raw = pd.DataFrame() # Global raw variable
comp_count = url_count(query, "investments") 
data_acq = 0 # Starting loop value

while data_acq < comp_count:
    if data_acq != 0: # Query loop
        #print("before0:", data_acq)
        last_uuid = raw.uuid[len(raw.uuid)-1] # selects the most recently added result
        query["after_id"] = last_uuid # saves most recent uuid query so POST request starts after this one
        url_extraction(query, "investments") # extracts data  
        data_acq = len(raw.uuid) # updates data_acq variable
        #print("after0:",data_acq)
    else:
        if "after_id" in query: # Removes after_id in case its there before the query starts.
            #print("before1:", data_acq)
            query = query.pop("after_id")
            url_extraction(query, "investments")
            data_acq = len(raw.uuid)
            #print("after1:",data_acq)
        else: # Starting query loop
            #print("before2:", data_acq)
            url_extraction(query, "investments")
            data_acq = len(raw.uuid)
            #print("after2:", data_acq)